In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel
import time
import os
import json
from typing import List, Tuple, Dict, Any
from datetime import datetime

# Fungsi ini harus didefinisikan di scope global agar bisa dipanggil oleh kelas.
def custom_log(level: str, message: str):
    """
    Mencetak pesan log ke konsol dengan format waktu, level, dan pesan yang seragam.
    
    Args:
        level (str): Tingkat keparahan log (misalnya "INFO", "ERROR", "WARNING", "CRITICAL").
        message (str): Pesan log yang akan dicetak.
    """
    # Mengambil waktu saat ini dan memformatnya
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Mencetak output log
    print(f"{timestamp} - {level} - {message}")

# --- Configuration ---
PROJECT_ID = "talk-of-the-heart-aac-464912"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.5-pro"
INPUT_FILE_NAME = "Dataset_Key.txt"
OUTPUT_FILE_NAME = "Dataset_TOTH.jsonl"
CHECKPOINT_FILE_NAME = "Dataset_checkpoint.json"
BATCH_SIZE = 150
MAX_RETRIES = 5
INITIAL_DELAY_SECONDS = 0.5
MAX_DELAY_SECONDS = 10

# --- Vertex Initiliaze ---
# Fungsi untuk menginisialisasi dan mengkonfigurasi koneksi ke Vertex AI.
def initialize_vertex_ai(project_id: str, location: str, model_name: str) -> GenerativeModel:
    try:
        custom_log("INFO", f"Initializing Vertex AI for project '{project_id}' in location '{location}'...")

        # Inisialisasi SDK Vertex AI dengan Project ID dan Location yang ditentukan.
        vertexai.init(project=project_id, location=location)

        # Memuat model Generative (Gemini)
        model = GenerativeModel(model_name)
        custom_log("INFO", f"Successfully configured Generative Model '{model_name}'.")
        return model
    except Exception as e:
        custom_log("CRITICAL", f"Gagal mengkonfigurasi Vertex AI. Kesalahan: {e}")
        custom_log("CRITICAL", "Pastikan kredensial ADC (gcloud auth application-default login) dan Project ID sudah benar.")
        raise # Menghentikan eksekusi jika inisialisasi gagal.

# --- Core Logic ---
# Fungsi untuk meminta model Gemini mengubah urutan kata kunci menjadi kalimat sempurna.
def generate_complete_sentences(keywords_sequence: List[str], model: GenerativeModel) -> List[Tuple[str, str]]:
    
    # Memformat daftar urutan kata kunci menjadi string yang dipisahkan baris untuk prompt LLM.
    formatted_keys_input = "\n".join(keywords_sequence)
    current_batch_size = len(keywords_sequence)

    # Definisi Prompt Model: Instruksi rinci dan ketat untuk model Gemini.
    llm_prompt = f"""
    ROLE: Anda adalah **Ahli Bahasa Indonesia dan Editor Senior** yang sangat teliti. Tugas Anda adalah mengubah setiap urutan kata kunci (KEY) menjadi satu kalimat Bahasa Indonesia (VALUE) yang 100% sempurna, konsisten, dan natural. Kualitas dan konsistensi adalah segalanya.

    ---
    ATURAN LINGUISTIK (SANGAT KRITIS)

    A. ATURAN KATA KERJA & WAKTU (ASPEK):
        1. **Imbuhan Aktif:** SELALU ubah kata kerja dasar (`lihat`, `main`, `gunting`) menjadi bentuk aktif berimbuhan (`melihat`, `bermain`, `menggunting`).
        2. **Aksi Berlangsung ('Sedang'):** JIKA kalimat adalah aksi aktif DAN TIDAK ADA keterangan waktu/lagi/sekarang, MAKA WAJIB gunakan kata **'sedang'**.
        3. **Aksi Berulang ('Lagi'):** JIKA ada kata 'lagi', JANGAN gunakan 'sedang' dan letakkan 'lagi' di akhir kalimat.
        4. **Waktu (Pagi/Siang/Malam):** JANGAN gunakan 'sedang'. WAJIB gunakan frasa **'pada [waktu] hari'**.

    B. ATURAN KONDISI & DESKRIPSI:
        5. **Perasaan/Status (Lapar, Sedih, Haus, Senang):** WAJIB awali dengan kata **'merasa'**.
        6. **Deskripsi Benda (`[benda], [sifat]`):** WAJIB gunakan kata **'terlihat'** atau **'terasa'** untuk deskripsi. SANGAT DILARANG menggunakan pola 'Buku itu tebal' atau 'Piring bersih'.

    C. ATURAN STRUKTUR & KONEKTOR:
        7. **Kata Sambung 'Dan':** WAJIB gunakan **'dan'** untuk menghubungkan kondisi/niat (e.g., merasa lapar dan mau makan) atau aksi/objek berganda.
        8. **Sebab-Akibat:** JIKA konteks menyiratkan kausalitas (e.g., `marah`, `kotor`), WAJIB gunakan kata **'karena'**.
        9. **Alat (Instrumen):** JIKA kata kunci pertama adalah alat (`sumpit`), balik struktur menjadi **`[Aksi] menggunakan [Alat]`**.
        10. **Negasi:** Letakkan kata **'tidak'** SEBELUM kata kerja/sifat yang dinegasikan (e.g., tidak merasa lapar).

    D. ATURAN KATA SPESIFIK:
        11. **Kata 'Cinta':** WAJIB diubah menjadi **'mencintai'**. DILARANG KERAS menggantinya dengan sinonim seperti 'menyukai'.

    ---
    STANDAR EMAS & ANTI-POLA (PELAJARI CONTOH-CONTOH INI!)
    **TEMA: KETERANGAN WAKTU**
    - BENAR: "adik laki-laki, tidur, siang" -> Adik laki-laki sedang tidur pada siang hari.
    - SALAH: Adik laki-laki tidur siang.

    **TEMA: PENGGUNAAN "SEDANG"**
    - BENAR: "adik laki-laki, makan, pisang" -> Adik laki-laki sedang makan pisang.
    - SALAH: Adik laki-laki makan pisang.

    **TEMA: DESKRIPSI BENDA**
    - BENAR: "piring, bersih" -> Piring terlihat bersih.
    - SALAH: Piring bersih.

    **TEMA: SEBAB-AKIBAT**
    - BENAR: "ibu, marah, adik laki-laki, kotor" -> Ibu marah karena adik laki-laki terlihat kotor.
    - SALAH: Ibu marah, adik laki-laki kotor.

    **TEMA: NEGASI PERASAAN**
    - BENAR: "kamu, lapar, tidak" -> Kamu tidak merasa lapar.
    - SALAH: Kamu merasa lapar tidak.

    ---
    DATA INPUT (Urutan Kata Kunci Bahasa Indonesia):
    {formatted_keys_input}
    ---
    FORMAT OUTPUT WAJIB:
    [Urutan Kata Kunci Asli] | [Kalimat Bahasa Indonesia Lengkap dan Natural]
    ---
    MULAI GENERASI OUTPUT:
    """

    delay = INITIAL_DELAY_SECONDS
    # Logika percobaan ulang (retry) dengan peningkatan jeda (exponential backoff).
    for attempt in range(MAX_RETRIES):
        try:
            # Memanggil API Gemini.
            response = model.generate_content(llm_prompt)
            raw_results = response.text.strip().split('\n')

            parsed_output_batch = []
            # Memproses dan memverifikasi setiap baris hasil LLM.
            for line in raw_results:
                if '|' in line:
                    parts = line.split('|', 1)
                    if len(parts) == 2:
                        key_str, value_str = parts
                        # Memastikan KEY dan VALUE tidak kosong
                        if key_str.strip() and value_str.strip():
                            parsed_output_batch.append((key_str.strip(), value_str.strip()))

            # Pengecekan Kritis: Apakah jumlah output sesuai dengan jumlah input?
            if current_batch_size == len(parsed_output_batch):
                custom_log("INFO", f"✅ Berhasil: Generasi {len(parsed_output_batch)} kalimat dari batch.")
                return parsed_output_batch # Berhasil, kembalikan batch yang sudah diproses.
            else:
                custom_log("WARNING", f"⚠️ Peringatan: Jumlah output ({len(parsed_output_batch)}) tidak sesuai dengan input ({current_batch_size}). Coba lagi...")
                # Lanjut ke logika penundaan dan retry.

        except Exception as e:
            custom_log("ERROR", f"❌ Kesalahan API pada percobaan {attempt + 1}/{MAX_RETRIES}: {e}")
            # Lanjut ke logika penundaan dan retry.

        # Logika Exponential Backoff: Jeda digandakan setiap kali gagal, maksimal 10 detik.
        delay = min(delay * 2, MAX_DELAY_SECONDS)
        custom_log("INFO", f"Menunggu {delay} detik sebelum mencoba lagi...")
        time.sleep(delay)

    custom_log("ERROR", f"Gagal memproses batch setelah {MAX_RETRIES} percobaan.")
    return [] # Mengembalikan daftar kosong jika semua percobaan gagal.

# Fungsi untuk menggabungkan data baru ke data yang sudah ada, menangani duplikat (mengganti yang lama).
def remove_duplicates_and_merge(existing_data: Dict[str, str], new_data: List[Tuple[str, str]]) -> Dict[str, str]:
    # Menggunakan dictionary, di mana KEYWORDS sequence adalah kuncinya (otomatis menangani duplikat)
    for key, value in new_data:
        existing_data[key] = value
    return existing_data

# Fungsi untuk menyimpan progres saat ini ke file checkpoint.
def save_checkpoint(data: Dict[str, str], processed_count: int):
    checkpoint_data = {
        "processed_count": processed_count, # Jumlah total baris yang sudah diproses.
        "data": data # Semua data KEY:VALUE yang sudah terkumpul.
    }
    # Menyimpan sebagai JSON yang diformat agar mudah dibaca.
    with open(CHECKPOINT_FILE_NAME, "w", encoding="utf-8") as f:
        json.dump(checkpoint_data, f, ensure_ascii=False, indent=4)
    custom_log("INFO", f"Checkpoint disimpan: {processed_count} baris telah diproses.")

# Fungsi untuk memuat data dan status progres dari file checkpoint.
def load_checkpoint() -> Tuple[Dict[str, str], int]:
    if os.path.exists(CHECKPOINT_FILE_NAME):
        try:
            with open(CHECKPOINT_FILE_NAME, "r", encoding="utf-8") as f:
                checkpoint_data = json.load(f)
            custom_log("INFO", f"Checkpoint ditemukan. Melanjutkan dari {checkpoint_data['processed_count']} baris.")
            # Mengembalikan data yang sudah ada dan indeks baris terakhir yang diproses.
            return checkpoint_data["data"], checkpoint_data["processed_count"]
        except (json.JSONDecodeError, KeyError) as e:
            custom_log("WARNING", f"File checkpoint rusak atau format salah: {e}. Memulai dari awal.")
    return {}, 0 # Mengembalikan dictionary kosong dan indeks 0 jika checkpoint gagal atau tidak ada.

# Fungsi utama yang mengorkestrasi seluruh proses.
def process_and_save_dataset():
    
    # Inisialisasi model Gemini.
    try:
        gemini_model = initialize_vertex_ai(PROJECT_ID, LOCATION, MODEL_NAME)
    except Exception:
        return # Keluar jika inisialisasi kritis gagal.

    custom_log("INFO", f"Memulai proses untuk menghasilkan kalimat dari {INPUT_FILE_NAME}...")

    # Membaca semua urutan kata kunci dari file input.
    try:
        with open(INPUT_FILE_NAME, 'r', encoding='utf-8') as f:
            all_keywords_sequences = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        custom_log("CRITICAL", f"❌ Kesalahan: File input '{INPUT_FILE_NAME}' tidak ditemukan.")
        return

    # Memuat data dari checkpoint.
    final_data, start_index = load_checkpoint()
    total_sequences = len(all_keywords_sequences)

    # Memeriksa apakah semua data sudah selesai diproses.
    if start_index >= total_sequences:
        custom_log("INFO", "Semua data sudah diproses berdasarkan checkpoint.")
    else:
        custom_log("INFO", f"Total urutan kata kunci yang dibaca: {total_sequences}")

        # Perulangan untuk memproses data dalam batch.
        for i in range(start_index, total_sequences, BATCH_SIZE):
            current_batch_sequences = all_keywords_sequences[i:i + BATCH_SIZE]

            # Menghitung dan mencetak progres batch.
            total_batches = (total_sequences + BATCH_SIZE - 1) // BATCH_SIZE
            current_batch_num = i // BATCH_SIZE + 1
            custom_log("INFO", f"\nMemproses Batch {current_batch_num}/{total_batches} (Baris {i+1} hingga {i + len(current_batch_sequences)} dari {total_sequences})")

            # Memanggil LLM untuk menghasilkan kalimat.
            generated_batch = generate_complete_sentences(current_batch_sequences, gemini_model)

            # Menyimpan data yang berhasil dan memperbarui checkpoint.
            if generated_batch:
                final_data = remove_duplicates_and_merge(final_data, generated_batch)
                processed_count = i + len(current_batch_sequences)
                save_checkpoint(final_data, processed_count)
            else:
                custom_log("ERROR", f"Gagal memproses batch yang dimulai dari baris {i+1}. Proses dihentikan.")
                break # Menghentikan perulangan jika satu batch gagal total.

    # --- PENYIMPANAN DATASET FINAL ---
    print("\n--- PROSES SELESAI, MENYIMPAN DATASET FINAL ---")
    final_unique_entries = list(final_data.items())

    # Menyimpan data akhir ke format JSON Lines (JSONL).
    with open(OUTPUT_FILE_NAME, "w", encoding="utf-8") as f:
        for key_str, value_str in final_unique_entries:
            # Membersihkan string kunci dari karakter yang tidak diinginkan.
            temp_key = key_str.replace('"', '').replace("'", "").strip()
            clean_key_for_json = temp_key.lstrip('*- \t').strip()

            # Membentuk objek JSONL untuk setiap entri.
            data_entry = {
                "input_text": clean_key_for_json,
                "target_text": value_str
            }
            # Menulis JSON entry per baris.
            f.write(json.dumps(data_entry, ensure_ascii=False) + "\n")

    # Log dan pembersihan akhir.
    custom_log("INFO", f"✅ SELESAI! Total {len(final_unique_entries)} entri unik disimpan ke '{OUTPUT_FILE_NAME}' dalam format JSONL.")
    if os.path.exists(CHECKPOINT_FILE_NAME):
        os.remove(CHECKPOINT_FILE_NAME)
        custom_log("INFO", "File checkpoint telah dihapus.")

# Memastikan fungsi utama dijalankan saat skrip dieksekusi.
if __name__ == "__main__":
    process_and_save_dataset()


2025-09-26 07:05:47 - INFO - Initializing Vertex AI for project 'talk-of-the-heart-aac-464912' in location 'us-central1'...
2025-09-26 07:05:47 - INFO - Successfully configured Generative Model 'gemini-2.5-pro'.
2025-09-26 07:05:47 - INFO - Memulai proses untuk menghasilkan kalimat dari Dataset_Key.txt...
2025-09-26 07:05:47 - INFO - Total urutan kata kunci yang dibaca: 6897
2025-09-26 07:05:47 - INFO - 
Memproses Batch 1/46 (Baris 1 hingga 150 dari 6897)


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


2025-09-26 07:07:39 - INFO - ✅ Berhasil: Generasi 150 kalimat dari batch.
2025-09-26 07:07:39 - INFO - Checkpoint disimpan: 150 baris telah diproses.
2025-09-26 07:07:39 - INFO - 
Memproses Batch 2/46 (Baris 151 hingga 300 dari 6897)
2025-09-26 07:09:34 - INFO - ✅ Berhasil: Generasi 150 kalimat dari batch.
2025-09-26 07:09:34 - INFO - Checkpoint disimpan: 300 baris telah diproses.
2025-09-26 07:09:34 - INFO - 
Memproses Batch 3/46 (Baris 301 hingga 450 dari 6897)
2025-09-26 07:11:09 - INFO - ✅ Berhasil: Generasi 150 kalimat dari batch.
2025-09-26 07:11:09 - INFO - Checkpoint disimpan: 450 baris telah diproses.
2025-09-26 07:11:09 - INFO - 
Memproses Batch 4/46 (Baris 451 hingga 600 dari 6897)
2025-09-26 07:11:10 - ERROR - ❌ Kesalahan API pada percobaan 1/5: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.
2025-09-26 07:11:10 - INFO - Menunggu 1.0 detik sebelum mencoba lagi...
2025-09-2